In [1]:
# Importing required libraries
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from google.colab import drive
import sklearn.datasets as ds
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score
from sklearn.multioutput import MultiOutputRegressor
import random
random.seed(0)

In [122]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!ls "/content/drive/My Drive/"

'  (1).txt'
 29A69B82-1F56-4160-B054-188D16CA9E1F.png
 2-Amortized-Analysis.gdoc
'  (2).txt'
 3366222C-7941-4393-B30D-FA95CD1D817A.png
 ACN
 Assign6.gdoc
 Attendance.gsheet
 Chapter_1.gslides
'Chapter_2 (1).gslides'
'Chapter_2 (2).gslides'
'Chapter_2 (3).gslides'
'Chapter_2 (4).gslides'
'Chapter_2 (5).gslides'
'Chapter_2 (6).gslides'
'Chapter_2 (7).gslides'
 Chapter_2.gslides
'Chapter_3 (1).gslides'
'Chapter_3 (2).gslides'
'Chapter_3 (3).gslides'
'Chapter_3 (4).gslides'
 Chapter_3.gslides
'Chapter_4 (1).gslides'
'Chapter_4 (2).gslides'
'Chapter_4 (3).gslides'
'Chapter_4 (4).gslides'
'Chapter_4 (5).gslides'
'Chapter_4 (6).gslides'
'Chapter_4 (7).gslides'
'Chapter_4 (8).gslides'
'Chapter_4 (9).gslides'
 Chapter_4.gslides
'Chapter_5 (1).gslides'
'Chapter_5 (2).gslides'
'Chapter_5 (3).gslides'
'Chapter_5 (4).gslides'
'Chapter_5 (5).gslides'
 Chapter_5.gslides
 Classroom
'Colab Notebooks'
'Copy of 20220918_230353.jpg'
'CS2233-Data Structures- Marks List.gsheet'
 CS22MTECH11017_A1_Wireshark.

1. Repeat Q1 in section 2.2 above with l2 regularized logistic regression and
SVM. Also, normalize the entire dataset using https://scikit-learn.
org/stable/modules/generated/sklearn.preprocessing.normalize.
html?highlight=normalize#sklearn.preprocessing.normalize and use
only the normalized version for train-test split etc. Tune the regulariza-
tion hyperparameter, C = 0:01; 0:1; 1; 10; 100, using 3-fold cross-validation
repeated 5 times. Compare these accuracies with those obtained by you
earlier with (unregularized) logistic regression.

In [4]:
#read xlsx file
data = pd.read_excel("drive/My Drive/ML/LSVT_voice_rehabilitation.xlsx")
data.head(5)

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,...,det_TKEO_std4_1_coef,det_TKEO_std4_2_coef,det_TKEO_std4_3_coef,det_TKEO_std4_4_coef,det_TKEO_std4_5_coef,det_TKEO_std4_6_coef,det_TKEO_std4_7_coef,det_TKEO_std4_8_coef,det_TKEO_std4_9_coef,det_TKEO_std4_10_coef
0,0.088112,0.041697,0.000480,-3.723304e-06,0.000422,2.458381,6.332164e-07,47.021079,1366.430390,-7.103323,...,2.527583,7.088978,19.753255,54.335046,145.528630,375.097397,921.296579,2137.079844,4697.131077,9931.208257
1,0.161798,0.057364,0.000677,5.466365e-06,0.000206,2.592066,7.228518e-07,93.557936,2582.922776,-23.284761,...,2.841881,7.977363,22.203504,60.993338,163.560972,421.010306,1036.092589,2404.072562,5284.082128,11165.095662
2,0.554508,0.642913,0.007576,-7.443871e-07,0.006488,12.691326,6.946246e-04,52.988422,466.682635,-45.308680,...,1.806103,5.078616,14.135923,38.641654,103.466808,264.654626,649.657090,1507.384591,3315.804236,6974.600636
3,0.031089,0.027108,0.000314,-2.214722e-07,0.000216,0.754288,1.868647e-07,13.982754,417.217249,-1.207741,...,1.999637,5.610448,15.626164,42.943275,115.014975,296.320795,728.284936,1689.586636,3713.818933,7851.139360
4,0.076177,0.039071,0.000302,2.732106e-05,0.001102,1.270034,4.918186e-05,56.373996,1608.317410,-3.491990,...,2.453087,6.902199,19.117609,52.715873,141.113865,363.511021,893.246151,2071.625622,4554.204815,9623.566242


In [5]:
data.shape

(126, 310)

In [6]:
#normalise the data
data_normalized = preprocessing.normalize(data, norm='l2')

In [7]:
data_normalized

array([[5.62318332e-12, 2.66105937e-12, 3.06557559e-14, ...,
        1.36384869e-07, 2.99763066e-07, 6.33793137e-07],
       [5.56338201e-12, 1.97246793e-12, 2.32801877e-14, ...,
        8.26636029e-08, 1.81692214e-07, 3.83909807e-07],
       [2.33972023e-10, 2.71274225e-10, 3.19649870e-12, ...,
        6.36034117e-07, 1.39908861e-06, 2.94290122e-06],
       ...,
       [9.76820584e-12, 6.82416920e-12, 8.03280834e-14, ...,
        2.48205479e-07, 5.45502890e-07, 1.15429534e-06],
       [7.04635388e-12, 3.43178651e-12, 4.08451046e-14, ...,
        1.35881364e-07, 2.98481632e-07, 6.30638944e-07],
       [1.94564882e-11, 1.00475989e-11, 1.19366435e-13, ...,
        1.55409254e-07, 3.41614813e-07, 7.22523980e-07]])

In [8]:
#df1 = pd.read_excel(data, 'Data')


In [9]:
#df2 = pd.read_excel(data, 'Binary response')

In [10]:
df1 = pd.read_excel("drive/My Drive/ML/LSVT_voice_rehabilitation.xlsx", 0)
#df1

In [11]:
#df1_normalized = preprocessing.normalize(df1, norm='l2')

In [12]:
df2 = pd.read_excel("drive/My Drive/ML/LSVT_voice_rehabilitation.xlsx", 1)
#df2

In [13]:
#df2_normalized = preprocessing.normalize(df2, norm='l2')

In [14]:
df1.shape

(126, 310)

In [15]:
df2.shape

(126, 1)

In [16]:
#split data into train and test
'''
x_train, x_test, y_train, y_test = train_test_split(df1, df2, test_size = 0.5, random_state = 42)
print("x_train.shape :", x_train.shape)
print("x_test.shape :", x_test.shape)
print("y_train.shape :", y_train.shape)
print("y_test.shape :", y_test.shape)
'''

'\nx_train, x_test, y_train, y_test = train_test_split(df1, df2, test_size = 0.5, random_state = 42)\nprint("x_train.shape :", x_train.shape)\nprint("x_test.shape :", x_test.shape)\nprint("y_train.shape :", y_train.shape)\nprint("y_test.shape :", y_test.shape)\n'

In [17]:
X = df1.to_numpy()
Y = df2.to_numpy()

In [18]:
np.random.seed(0)
s = np.array([i for i in range(df1.shape[0])])
np.random.shuffle(s)
train_index = s[:63]
test_index = s[63:]
# check that there's no overlap
np.intersect1d(train_index, test_index)

array([], dtype=int64)

In [19]:
print(len(train_index))
print(len(test_index))

63
63


In [20]:
x_train = X[train_index]
y_train = Y[train_index].ravel()

x_test = X[test_index]
y_test = Y[test_index].ravel()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((63, 310), (63,), (63, 310), (63,))

In [21]:
"""
NOTE: This was not asked in the question but this helped Logistic regression converge.
"""
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

Using l2 regularized logistic regression 
https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression. C is the hyperparameter to be cross-validated

In [22]:
#using logistic regression
from sklearn.linear_model import LogisticRegression

#using cross validation
#from sklearn.model_selection import cross_validate

Reference used by me: https://medium.com/@mandava807/cross-validation-and-hyperparameter-tuning-in-python-65cfb80ee485

In [23]:
#KFold
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kfold = model_selection.RepeatedKFold(n_splits = 3, n_repeats = 5)

In [24]:
#case1: modeling with C=0.01
logiReg1 = LogisticRegression(penalty='l2', C= 0.01 )

In [25]:
#cross validation 
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

result1 = model_selection.cross_val_score(logiReg1, x_train, y_train, cv = kfold)


In [26]:
#check accuracy
print("Accuracy using C=0.01 is :", result1.mean()*100)

Accuracy using C=0.01 is : 76.50793650793652


In [27]:
#case2: modeling with C =0.1
logiReg2 = LogisticRegression(penalty='l2', C= 0.1 )
result2 = model_selection.cross_val_score(logiReg2, x_train, y_train, cv = kfold)
#result2 = cross_validate(logiReg2, x_train, y_train, cv = kfold)


In [28]:
print("Accuracy using C=0.1 is :", result2.mean()*100)

Accuracy using C=0.1 is : 73.96825396825395


In [29]:
#case3: with C =1
logiReg3 = LogisticRegression(penalty='l2', C= 1 )
result3 = model_selection.cross_val_score(logiReg3, x_train, y_train, cv = kfold)


In [30]:
print("Accuracy using C=1 is :", result3.mean()*100)

Accuracy using C=1 is : 74.28571428571428


In [31]:
#case 4 : C=10
logiReg4 = LogisticRegression(penalty='l2', C= 10 )
result4 = model_selection.cross_val_score(logiReg4, x_train, y_train, cv = kfold)

In [32]:
print("Accuracy using C=10 is :", result4.mean()*100)

Accuracy using C=10 is : 75.55555555555553


In [33]:
#case5 : C=100
logiReg5 = LogisticRegression(penalty='l2', C= 100)
result5 = model_selection.cross_val_score(logiReg5, x_train, y_train, cv = kfold)

In [34]:
print("Accuracy using C=100 is :", result5.mean()*100)

Accuracy using C=100 is : 76.82539682539684


In [35]:
#highest accuracy is obtained at C=10 i;e for logiReg4. 
logiReg4.fit(x_train, y_train)

LogisticRegression(C=10)

In [36]:
#make prediction
y_pred_log = logiReg4.predict(x_test)

In [37]:
print("Accuracy using LogisticReg(when C= 10) is: {0:.4f}%".format(accuracy_score(y_test, y_pred_log)*100))

Accuracy using LogisticReg(when C= 10) is: 82.5397%


**Now, Phi(x)=[1 x x^2]**


In [94]:
#transforming x into feature map i.e: phi(x) 
x_train_phi = np.concatenate((np.ones((63,1)), x_train, x_train**2), axis=1)
x_test_phi = np.concatenate((np.ones((63,1)), x_test, x_test**2), axis=1)

In [95]:
#check shape
print("shape of x_train_phi:",x_train_phi.shape)
print("shape of x_test_phi:",x_test_phi.shape)

shape of x_train_phi: (63, 621)
shape of x_test_phi: (63, 621)


In [99]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [100]:
x_train_phi = sc.fit_transform(x_train_phi)
x_test_phi = sc.fit_transform(x_test_phi)

In [101]:
#highest accuracy is obtained at C=10 i;e for logiReg4. 
logiReg4.fit(x_train_phi, y_train)

LogisticRegression(C=10)

In [102]:
#make prediction
y_pred_log_phi = logiReg4.predict(x_test_phi)

In [103]:
print("Accuracy using LogisticReg(when C= 10) is: {0:.4f}%".format(accuracy_score(y_test, y_pred_log_phi)*100))

Accuracy using LogisticReg(when C= 10) is: 88.8889%


**SVM**

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

In [38]:
#import package
from sklearn.svm import LinearSVC

In [39]:
lsvc1 = LinearSVC(penalty = 'l2', loss = 'hinge', C=0.01)

In [40]:
result1 = model_selection.cross_val_score(lsvc1, x_train, y_train, cv = kfold)

In [41]:
print("Accuracy using C=0.01 and SVM is :", result1.mean()*100)

Accuracy using C=0.01 and SVM is : 70.7936507936508


In [42]:
#case 2: when C =0.1
lsvc2 = LinearSVC(penalty = 'l2', loss = 'hinge', C=0.1)
result2 = model_selection.cross_val_score(lsvc2, x_train, y_train, cv = kfold)


In [43]:
print("Accuracy using C=0.1 and SVM is :", result2.mean()*100)

Accuracy using C=0.1 and SVM is : 64.44444444444444


In [44]:
#case3 : when C=1
lsvc3 = LinearSVC(penalty = 'l2', loss = 'hinge', C=1)
result3 = model_selection.cross_val_score(lsvc3, x_train, y_train, cv = kfold)

In [45]:
print("Accuracy using C=1 and SVM is :", result3.mean()*100)

Accuracy using C=1 and SVM is : 65.39682539682539


In [46]:
#case4 : when C=10
lsvc4 = LinearSVC(penalty = 'l2', loss = 'hinge', C=10)
result4 = model_selection.cross_val_score(lsvc4, x_train, y_train, cv = kfold)

In [47]:
print("Accuracy using C= 10 and SVM is :", result4.mean()*100)

Accuracy using C= 10 and SVM is : 67.93650793650792


In [48]:
#case 5: when C=100
lsvc5 = LinearSVC(penalty = 'l2', loss = 'hinge', C=100)
result5 = model_selection.cross_val_score(lsvc5, x_train, y_train, cv = kfold)

In [49]:
print("Accuracy using C= 100 and SVM is :", result5.mean()*100)

Accuracy using C= 100 and SVM is : 71.42857142857142


In [50]:
#highest accuracy obtained  using C=1 and SVM is : 71.74603174603172
lsvc3.fit(x_train, y_train)

LinearSVC(C=1, loss='hinge')

In [51]:
#make prediction
y_pred_svc = lsvc3.predict(x_test)

In [52]:
print("Accuracy using SVM: {0:.4f}%".format(accuracy_score(y_test, y_pred_svc)*100))

Accuracy using SVM: 84.1270%


**Now, Phi(x)=[1 x x^2]**


In [109]:
#highest accuracy obtained  using C=1 and SVM is : 71.74603174603172
lsvc3.fit(x_train_phi, y_train)

LinearSVC(C=1, loss='hinge')

In [110]:
#make prediction
y_pred_svc_phi = lsvc3.predict(x_test_phi)

In [111]:
print("Accuracy using SVM: {0:.4f}%".format(accuracy_score(y_test, y_pred_svc_phi)*100))

Accuracy using SVM: 84.1270%


**2. Repeat Q3 in section 2.1 above with ridge-regression16 and SVR17. Also,
normalize the entire dataset using https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html?highlight=normalize\#sklearn.preprocessing.normalize and use only the nor-
malized version for train-test split etc. Tune the regularization hyperpa-
rameter from the range 0:01; 0:1; 1; 10; 100, using 3-fold cross-validation
repeated 5 times. Compare these explained variances with those obtained
by you earlier with (unregularized) linear regression**

References used:
https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html#sklearn.svm.LinearSVR

In [53]:
#importing mat file
img_data = scipy.io.loadmat("drive/My Drive/ML/DrivFace.mat")
drivFaceData = img_data['drivFaceD'][0]

In [54]:
X_raw = img_data['drivFaceD'][0][0][0]
print(X_raw.shape)

(606, 6400)


In [55]:
#reading dataset
label_data = pd.read_csv("drive/My Drive/ML/drivPoints.txt", sep=",")

In [56]:
#vstack() function is used to stack the sequence of input arrays vertically to make a single array
y = np.vstack((label_data["xF"], label_data["yF"], label_data["wF"], label_data["hF"])).T

In [57]:
X_raw_normalized = preprocessing.normalize(X_raw, norm='l2')

In [58]:
y_norm = preprocessing.normalize(y, norm='l2')

In [59]:
id_list = list(range(len(X_raw)))
split_ratio = 0.5
split_index = int(len(X_raw)*0.5)
random.shuffle(id_list)
id_train = id_list[0:split_index]
id_test = id_list[split_index:]

In [60]:
x_train1 = X_raw_normalized[id_train]
x_test1 = X_raw_normalized[id_test]
print(x_train1.shape)
print(x_test1.shape)

(303, 6400)
(303, 6400)


In [61]:
y_train1 = y_norm[id_train]
y_test1 = y_norm[id_test]
print(y_train1.shape)
print(y_test1.shape)

(303, 4)
(303, 4)


In [62]:
#splitting into train and test data
'''
x_train_lin, x_test_lin, y_train_lin, y_test_lin = train_test_split(X_raw_normalized, yF_norm, train_size = 0.5, random_state = 42)
print("x_train.shape :", x_train_lin.shape)
print("y_train.shape :", y_train_lin.shape)
print("x_test.shape :", x_test_lin.shape)
print("y_test.shape :", y_test_lin.shape)
'''

'\nx_train_lin, x_test_lin, y_train_lin, y_test_lin = train_test_split(X_raw_normalized, yF_norm, train_size = 0.5, random_state = 42)\nprint("x_train.shape :", x_train_lin.shape)\nprint("y_train.shape :", y_train_lin.shape)\nprint("x_test.shape :", x_test_lin.shape)\nprint("y_test.shape :", y_test_lin.shape)\n'

In [63]:
#x_train_lin.dtype

In [64]:
#y_train_lin.dtype

**Ridge Regression**

In [65]:
from sklearn.linear_model import Ridge

In [66]:
#list storing C values
#C = [0.01, 0.1, 1, 10, 100]

In [67]:
cv_kfold = model_selection.RepeatedKFold(n_splits = 3, n_repeats = 5)

In [68]:
#case1 : when C=0.01
ridgeReg1 = Ridge(alpha = 0.01)
resultRidge1 = model_selection.cross_val_score(ridgeReg1, x_train1, y_train1, cv = cv_kfold)

In [69]:
print("Accuracy using C=0.01 and RidgeReg is :", resultRidge1.mean()*100)

Accuracy using C=0.01 and RidgeReg is : 63.47942609303363


In [70]:
#case2 : when C=0.1
ridgeReg2 = Ridge(alpha = 0.1)
resultRidge2 = model_selection.cross_val_score(ridgeReg2, x_train1, y_train1, cv = cv_kfold)

In [71]:
print("Accuracy using C=0.1 and RidgeReg is :", resultRidge2.mean()*100)

Accuracy using C=0.1 and RidgeReg is : 54.110305283350954


In [72]:
#case 3: when C=1
ridgeReg3 = Ridge(alpha = 1)
resultRidge3 = model_selection.cross_val_score(ridgeReg3, x_train1, y_train1, cv = cv_kfold)

In [73]:
print("Accuracy using C=1 and RidgeReg is :", resultRidge3.mean()*100)

Accuracy using C=1 and RidgeReg is : 31.54488970772851


In [74]:
#case 4: when C=10
ridgeReg4 = Ridge(alpha = 10)
resultRidge4 = model_selection.cross_val_score(ridgeReg4, x_train1, y_train1, cv = cv_kfold)

In [75]:
print("Accuracy using C=10 and RidgeReg is :", resultRidge4.mean()*100)

Accuracy using C=10 and RidgeReg is : 5.721984056038574


In [76]:
#case5 : when C=100
ridgeReg5 = Ridge(alpha = 100)
resultRidge5 = model_selection.cross_val_score(ridgeReg5, x_train1, y_train1, cv = cv_kfold)

In [77]:
resultRidge5

array([-0.01123106, -0.00456599,  0.00356731,  0.00855572,  0.00703495,
        0.0080168 , -0.00212444,  0.00016133,  0.00537227,  0.00660256,
       -0.02613249, -0.01065734, -0.00642245, -0.01277451,  0.00192775])

In [78]:
print("Accuracy using C=100 and RidgeReg is :", resultRidge5.mean()*100)

Accuracy using C=100 and RidgeReg is : -0.21779733541165544


In [79]:
#testing using best accuracy obtained at  C=0.01 and RidgeReg is : 61.081404443130424
ridgeReg1.fit(x_train1, y_train1)

Ridge(alpha=0.01)

In [80]:
y_pred1=ridgeReg1.predict(x_test1)

In [81]:
print("Explained variance for xF: ", explained_variance_score(y_test1[:,0], y_pred1[:,0]))
print("Explained variance for yF: ", explained_variance_score(y_test1[:,1], y_pred1[:,1]))
print("Explained variance for wF: ", explained_variance_score(y_test1[:,2], y_pred1[:,2]))
print("Explained variance for hF: ", explained_variance_score(y_test1[:,3], y_pred1[:,3]))
print("Explained variance for all: ", explained_variance_score(y_test1, y_pred1))

Explained variance for xF:  0.6780667585252533
Explained variance for yF:  0.6845556687916889
Explained variance for wF:  0.3879756832463035
Explained variance for hF:  0.524830340359413
Explained variance for all:  0.5688571127306644


**Now, Phi(x)=[1 x x^2]**


In [132]:
#transforming x into feature map i.e: phi(x) 
x_train1_phi = np.concatenate((np.ones((303,1)), x_train1, x_train1**2), axis=1)
x_test1_phi = np.concatenate((np.ones((303,1)), x_test1, x_test1**2), axis=1)

In [133]:
#check shape
print("shape of x_train1_phi:",x_train1_phi.shape)
print("shape of x_test1_phi:",x_test1_phi.shape)

shape of x_train1_phi: (303, 12801)
shape of x_test1_phi: (303, 12801)


In [134]:
x_train1_phi = sc.fit_transform(x_train1_phi)
x_test1_phi = sc.fit_transform(x_test1_phi)

In [135]:
#testing using best accuracy obtained at  C=0.01 and RidgeReg is : 61.081404443130424
ridgeReg1.fit(x_train1_phi, y_train1)

Ridge(alpha=0.01)

In [136]:
y_pred1_phi=ridgeReg1.predict(x_test1_phi)

In [137]:
print("Explained variance for xF: ", explained_variance_score(y_test1[:,0], y_pred1_phi[:,0]))
print("Explained variance for yF: ", explained_variance_score(y_test1[:,1], y_pred1_phi[:,1]))
print("Explained variance for wF: ", explained_variance_score(y_test1[:,2], y_pred1_phi[:,2]))
print("Explained variance for hF: ", explained_variance_score(y_test1[:,3], y_pred1_phi[:,3]))
print("Explained variance for all: ", explained_variance_score(y_test1, y_pred1_phi))

Explained variance for xF:  0.6257291003383851
Explained variance for yF:  0.6775513092565812
Explained variance for wF:  0.32226636615491555
Explained variance for hF:  0.4038344753435762
Explained variance for all:  0.5073453127733645


**SVR**

In [82]:
#import package
from sklearn.svm import LinearSVR

In [83]:
"""
NOTE: This was not asked in the question but this helped Linear regression converge.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train1)
x_train1 = scaler.transform(x_train1)
x_test1 = scaler.transform(x_test1)
"""

'\nNOTE: This was not asked in the question but this helped Linear regression converge.\n\nfrom sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nscaler.fit(x_train1)\nx_train1 = scaler.transform(x_train1)\nx_test1 = scaler.transform(x_test1)\n'

In [84]:
#case1 : when C=0.01
svr1 = MultiOutputRegressor(LinearSVR(C = 0.01))
resultsvr1 = model_selection.cross_val_score(svr1, x_train1, y_train1, cv = 3, scoring = 'neg_root_mean_squared_error')

In [85]:
resultsvr1

array([-0.04196967, -0.0423235 , -0.04250208])

In [86]:
print("Accuracy using C=0.01 and SVR is :", resultsvr1.mean()*100)

Accuracy using C=0.01 and SVR is : -4.226508282404877


In [87]:
svr2 =  MultiOutputRegressor(LinearSVR(C = 0.1))
resultsvr2 = model_selection.cross_val_score(svr2, x_train1, y_train1, cv = 3, scoring = 'neg_root_mean_squared_error')

In [88]:
print("Accuracy using C=0.1 and SVR is :", resultsvr2.mean()*100)

Accuracy using C=0.1 and SVR is : -3.08484993027601


In [89]:
svr3 =  MultiOutputRegressor(LinearSVR(C = 1,  max_iter=10000))
resultsvr3 = model_selection.cross_val_score(svr3, x_train1, y_train1, cv = 3, scoring = 'neg_root_mean_squared_error')

In [90]:
print("Accuracy using C=1 and SVR is :", resultsvr3.mean()*100)

Accuracy using C=1 and SVR is : -2.4829245180945483


In [91]:
svr4 =  MultiOutputRegressor(LinearSVR(C = 10,  max_iter=40000))
resultsvr4 = model_selection.cross_val_score(svr4, x_train1, y_train1, cv = 3, scoring = 'neg_root_mean_squared_error')

In [92]:
print("Accuracy using C=10 and SVR is :", resultsvr4.mean()*100)

Accuracy using C=10 and SVR is : -2.4367392288566676


In [123]:
svr5 =  MultiOutputRegressor(LinearSVR(C = 100,  max_iter=1000))
resultsvr5 = model_selection.cross_val_score(svr5, x_train1, y_train1, cv = 3, scoring = 'neg_root_mean_squared_error')

In [124]:
print("Accuracy using C=100 and SVR is :", resultsvr5.mean()*100)

Accuracy using C=100 and SVR is : -2.4330472501108593


In [125]:
#least mean square obtained at C=100
#testing using best accuracy obtained at  C=100 and SVR 
svr5.fit(x_train1, y_train1)

MultiOutputRegressor(estimator=LinearSVR(C=100))

In [126]:
y_pred1_svr=svr5.predict(x_test1)

In [127]:
print("Explained variance for xF: ", explained_variance_score(y_test1[:,0], y_pred1_svr[:,0]))
print("Explained variance for yF: ", explained_variance_score(y_test1[:,1], y_pred1_svr[:,1]))
print("Explained variance for wF: ", explained_variance_score(y_test1[:,2], y_pred1_svr[:,2]))
print("Explained variance for hF: ", explained_variance_score(y_test1[:,3], y_pred1_svr[:,3]))
print("Explained variance for all: ", explained_variance_score(y_test1, y_pred1_svr))

Explained variance for xF:  0.6398074701441697
Explained variance for yF:  0.6750639502905196
Explained variance for wF:  0.2826642673814427
Explained variance for hF:  0.4430560741145657
Explained variance for all:  0.5101479404826744


**Now, Phi(x)=[1 x x^2]**


In [ ]:
#transforming x into feature map i.e: phi(x) 
x_train_phi = np.concatenate((np.ones((63,1)), x_train, x_train**2), axis=1)
x_test_phi = np.concatenate((np.ones((63,1)), x_test, x_test**2), axis=1)

In [ ]:
#check shape
print("shape of x_train_phi:",x_train_phi.shape)
print("shape of x_test_phi:",x_test_phi.shape)

In [128]:
svr5.fit(x_train1_phi, y_train1)

MultiOutputRegressor(estimator=LinearSVR(C=100))

In [129]:
y_pred1_svr_phi=svr5.predict(x_test1_phi)

In [131]:
print("Explained variance for xF: ", explained_variance_score(y_test1[:,0], y_pred1_svr_phi[:,0]))
print("Explained variance for yF: ", explained_variance_score(y_test1[:,1], y_pred1_svr_phi[:,1]))
print("Explained variance for wF: ", explained_variance_score(y_test1[:,2], y_pred1_svr_phi[:,2]))
print("Explained variance for hF: ", explained_variance_score(y_test1[:,3], y_pred1_svr_phi[:,3]))
print("Explained variance for all: ", explained_variance_score(y_test1, y_pred1_svr_phi))

Explained variance for xF:  0.6263655856275194
Explained variance for yF:  0.6767506621956991
Explained variance for wF:  0.3228479235784991
Explained variance for hF:  0.404410938699972
Explained variance for all:  0.5075937775254223
